## MSOL ECE 231A : Data Compression Project Module 1

Please follow our instructions in the same order and print out the entire results and codes when completed.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from collections import Counter

In [3]:
##########################################################################################
# Read Text file
##########################################################################################
f = open("toy_example.txt", "r")
text = f.read()

In [5]:
text

'DACDDBCD'

In [10]:
##########################################################################################
# Compute the empirical distribution
##########################################################################################
def compute_distribution(text):
    """
    Inputs:
    - text: A string containing the text to be encoded.
    
    Returns:
    - symbols: a list of tuples of the form (char,prob), where char is a character appears in the text 
               and prob is the number of times this character appeared in text divided by the length of text.
    """
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #
    counts = Counter(text)
    n_chars = sum(counts.values())
    symbols = [
        (key, value / n_chars)
        for key, value 
        in counts.items()
    ]
    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #  
    return symbols

symbols = compute_distribution(text)
size_symbols = len(symbols)
print(text)
print(symbols)


DACDDBCD
[('D', 0.5), ('A', 0.125), ('C', 0.25), ('B', 0.125)]


## Part 1: Binary Huffman Codes

In [25]:
########################################################################################
# Draw the tree for the Huffman code 
########################################################################################
def Huffman_tree(symbols):
    """
    Inputs:
    - symbols: a list of tuples of the form (char,prob), where char is a character appears in the text 
               and prob is the number of times this character appeared in text divided by the length of text.
    
    Returns:
    - tree: a list of a single element that have probability one. at each iteration sort your list according 
            to their probabilities and combine the first two elements as a single element  
    """
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #
    def merge_chars(a, b):
        '''merge two (seq, prob) tuples'''
        return ((b[0],a[0]), a[1]+b[1])

    def tree_step(symbols):
        '''perform one step of huffman tree building by merging two lowest-probability chars'''
        symbols = symbols.copy()
        symbols.sort(key=lambda x: x[1])
        out = [merge_chars(*symbols[:2]), *symbols[2:]]
        return out

    tree = symbols
    steps = [symbols]
    while len(tree) > 1:
        tree = tree_step(tree)
        steps.insert(0, tree)

    print(steps)
        
    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return tree

tree = Huffman_tree(symbols)   
print(tree[0][0])

[[(('D', ('C', ('B', 'A'))), 1.0)], [(('C', ('B', 'A')), 0.5), ('D', 0.5)], [(('B', 'A'), 0.25), ('C', 0.25), ('D', 0.5)], [('A', 0.125), ('B', 0.125), ('C', 0.25), ('D', 0.5)]]
('D', ('C', ('B', 'A')))


In [26]:
#######################################################################################
#Encode the Huffman Tree
#######################################################################################
def Huffman_coding(seq, code=''):
    """
    Inputs:
    - seq: a tuple of characters.
    - code: the code of this tuple
    Returns:
    - Dictionary: a dictionary containing the Huffman codes.  
    """
    if type(seq) is str:
        return {seq : code}
    Dictionary = dict()
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #
    l_node = Huffman_coding(seq[0], code=code+'0')
    r_node = Huffman_coding(seq[1], code=code+'1')
    Dictionary = {**l_node, **r_node}

    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return Dictionary

Huffman_code = Huffman_coding(tree[0][0])
print("Huffman CodeBook:  ", Huffman_code)

Huffman CodeBook:   {'D': '0', 'C': '10', 'B': '110', 'A': '111'}


In [28]:
#######################################################################################
# Compute the expected length of the Huffman code
#######################################################################################
def compute_expected_length(symbols, Huffman_code):
    """
    Inputs:
    - symbols: A list of tuples of the form (char,prob).
    - Huffman_code: a dictionary containing the Huffman codes.Each code is a string
    Returns:
    - Expected_length: a number represents the expected length of Huffman code
    """
    
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #
    Expected_length = sum(
        sym[1]*len(Huffman_code[sym[0]]) 
        for sym in symbols
    )
    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return Expected_length

Expected_length = compute_expected_length(symbols, Huffman_code)
print("Expected length of Huffman code:   ", Expected_length)

Expected length of Huffman code:    1.75


In [29]:
#######################################################################################
# Encode a text
#######################################################################################
def encode_text(text, Huffman_code):
    """
    Inputs:
    - text: A string containing the text to be encoded.
    - Huffman_code: a dictionary containing the Huffman codes.Each code is a string
    Returns:
    - txt_code: a string represents the code of the input text.
    """
    txt_code = ''
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #
    for char in text:
        txt_code += Huffman_code[char]
    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return txt_code
txt_code = encode_text(text, Huffman_code)
print("Encoded Text:  ", txt_code) 

Encoded Text:   01111000110100


In [37]:
#######################################################################################
# Decode a text
#######################################################################################
def decode_text(txt_code, Huffman_code, symbols):
    """
    Inputs:
    -symbols: a list of symbols.
    - txt_code: A code of a text encoded by Huffman code as a string.
    - Huffman_code: a dictionary containing the Huffman codes. Each code is a string
    Returns:
    - decoded_text: a string represents the decoded text.
    """
    decoded_text = ''
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #
    Huffman_inverse = {b: a for a, b in Huffman_code.items()}
    txt_code = [*txt_code]
    temp_cw = ''
    while txt_code:
        temp_cw += txt_code.pop(0)
        temp_char = Huffman_inverse.get(temp_cw, None)
        if temp_char is not None:
            decoded_text+= temp_char
            temp_cw = ''

    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return decoded_text

decoded_text = decode_text(txt_code, Huffman_code, symbols)  
print("Orginal text:   ", text)      
print("Decoded Text:   ", decoded_text)

Orginal text:    DACDDBCD
Decoded Text:    DACDDBCD


## Part 2: Arithmetic Codes

In [ ]:
#######################################################################################
# Compute the expected length of the Huffman code
#######################################################################################
def compute_CDF(symbols):
    """
    Inputs:
    - symbols: A list of tuples of the form (char,prob).
    Returns:
    - CDF_symbols: A list of tuples of the form (char,CDF)
    """
    CDF_symbols = []
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #

    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return CDF_symbols

CDF_symbols = compute_CDF(symbols)
print(CDF_symbols)

In [ ]:
###########################################################################
# Decimal encoding
###########################################################################
def decimal_encoding(text,CDF_symbols) :
    """
    Inputs:
    - text: A string containing the text to be encoded.
    Returns:
    - lower: the lower value of the interval of the encoded text.
    - upper: the upper value of the interval of the encoded text.
    """
    lower = 0 
    upper = 1
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #
    for c in text:
    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return lower, upper

lower,upper = decimal_encoding(text,CDF_symbols)
print("Interval representing the text is:  ", lower, upper)

In [ ]:
###########################################################################
# Binary encoding
###########################################################################
def Arithmetic_encoding(lower,upper):
    """
    Inputs:
    - lower: the lower value of the interval of the encoded text.
    - upper: the upper value of the interval of the encoded text.
    Returns:
    - txt_code: a string represents the code of the input text.
    """
    txt_code = ''
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #

    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return txt_code

txt_code = Arithmetic_encoding(lower,upper)
print("Encoded Text:  ", txt_code)
Expected_length_Arithematic = len(txt_code)/len(text)
print("Expected length of Arithematic code:   ", Expected_length_Arithematic)

In [ ]:
###########################################################################
# Binary Decoding
###########################################################################
def decimal_decoding(txt_code):
    """
    Inputs:
    - txt_code: a string of zeros and ones represents the code of the input text.
    Returns:
    - decoded_val: a real number between 0 and 1.
    """
    decoded_val = 0
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #

    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return decoded_val

decoded_val = decimal_decoding(txt_code)
print("The decoded Value:  ", decoded_val)

In [ ]:
###########################################################################
# Arithmetic Decoding
###########################################################################
def Arithmetic_decode(decoded_val,CDF_symbols, n):
    """
    Inputs:
    - decoded_val: A real number between 0 and 1 represents the mid-point of the interval of the encoded text.
    - CDF_symbols: A list containing the symbols and their corresponding CDF
    - n: number of symbols to be decoded
    Returns:
    - decoded_text: a string containing the decoded text.
    """
    decoded_text = ''
    # ================================================================ #
    # YOUR CODE HERE:
    # ================================================================ #

    # ================================================================ #
    # END YOUR CODE HERE
    # ================================================================ #
    return decoded_text

decoded_text = Arithmetic_decode(decoded_val,CDF_symbols, len(text))
print("Orginal text:   ", text)      
print("Decoded Text:   ", decoded_text)